# Perform bootstrap to estimate stability of networks and network measures

In [1]:
from inverse_covariance import (
    QuicGraphicalLasso,
    QuicGraphicalLassoCV,
    QuicGraphicalLassoEBIC,
    AdaptiveGraphicalLasso,
    ModelAverage,
)

import sys
import numpy as np
import tabulate
import time

import logging
import copy

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
import matplotlib.pyplot as plt
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import scipy
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

from collections import defaultdict

from copy import deepcopy
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.utils import resample
from sklearn.covariance import GraphicalLassoCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

from itertools import combinations
from scipy.stats import ttest_ind

from scipy.stats import pearsonr
from scipy.spatial import distance
import scipy.stats as stats

import networkx as nx
import glob, os
from functools import partial

from scipy.stats import pearsonr

import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
def adaptive_model_average_sklearn(
    X, 
    method, penalization='random', 
    n_trials=100, 
    metric='log_likelihood', 
    support_thresh=0.5, 
    cv=10, 
    lam=None, 
    alphas=np.logspace(-5, 1, num=20)
):
    """Run ModelAverage in default mode (QuicGraphicalLassoCV) to obtain proportion
    matrix.

    NOTE:  Only method = 'binary' really makes sense in this case.
    """
    n_trials = n_trials

    if lam is None:
        cv_model = GraphicalLassoCV(
            alphas=alphas,
            cv=cv
        )
        cv_model.fit(X)
        lam = cv_model.alpha_
    else:
        lam = lam

    model = AdaptiveGraphicalLasso(
        estimator=ModelAverage(
            n_trials=n_trials, 
            penalization=penalization, 
            lam=lam, 
            n_jobs=10, 
            support_thresh=support_thresh,
            # subsample = 0.6,
        ),
        method=method,
    )
    model.fit(X)
    lam_norm_ = np.linalg.norm(model.estimator_.lam_)
    return model.estimator_.precision_

In [3]:
def learn_graph_structure_adaptive_average_sklearn(
    df, 
    n_trials=100, 
    penalization='random',
    score_metric="log_likelihood", 
    cv=10, 
    lam=None,
    threshold=0.5, 
    alphas=np.logspace(-5, 1, num=20)
):
    
    # standardize the time series: using correlations rather than covariance
    # former is more efficient for structure recovery
    X = df.to_numpy()
    X -= X.mean(axis=0)
    X /= X.std(axis=0)

    prec_adaptive = adaptive_model_average_sklearn(
        X, 
        penalization='random',
        method='binary', 
        n_trials=n_trials, 
        metric=score_metric,
        support_thresh=threshold,
        cv=cv,
        lam=lam,
        alphas=alphas,
    )
    
    precision_matrix_df = pd.DataFrame(prec_adaptive, columns = df.columns, index = df.columns)
            
    return precision_matrix_df

In [4]:
def estimate_graph(
    X, 
    n_trials=1000, 
    score_metric="log_likelihood", 
    cv=3, 
    threshold=0.65, 
    alphas=np.linspace(0.01, 0.1, 20)
):
    precision_matrix_df = learn_graph_structure_adaptive_average_sklearn(
            X, 
            penalization='random',
            n_trials=n_trials,
            score_metric=score_metric,
            cv=cv,
            lam=None,
            alphas=alphas,
            threshold=threshold,
    )

    this_links = get_links(precision_matrix_df)
    this_links['weight'] = this_links['weight'].abs()
    G_ = nx.from_pandas_edgelist(this_links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
    
    return G_, precision_matrix_df

## Read data

Define dataset

In [5]:
test = False
dataset = 'test' if test else 'train'
dataset

'train'

In [6]:
ern_data_df = pd.read_pickle(f"data/models_pickles_new/ern_models_{dataset}.pkl")
ern_cov_fal_data_df = pd.read_pickle(f"data/models_pickles_new/ern_cov_fal_models_{dataset}.pkl")

crn_data_df = pd.read_pickle(f"data/models_pickles_new/crn_models_{dataset}.pkl")
crn_cov_fal2_data_df = pd.read_pickle(f"data/models_pickles_new/crn_cov_fal2_models_{dataset}.pkl")

In [7]:
if test:
    display(crn_cov_fal2_data_df[crn_cov_fal2_data_df.isna().any(axis=1)])
    crn_cov_fal2_data_df['e_LT_F2_C'] = crn_cov_fal2_data_df['e_LT_F2_C'].fillna(crn_cov_fal2_data_df['e_LT_F2_C'].mean())
    display(crn_cov_fal2_data_df[crn_cov_fal2_data_df.isna().any(axis=1)])

In [8]:
datasets = [
    ern_data_df, 
    ern_cov_fal_data_df, 
    crn_data_df,
    crn_cov_fal2_data_df,
]

Read test data

In [9]:
ern_data_df_test = pd.read_pickle(f"data/models_pickles_new/ern_models_test.pkl")
ern_cov_fal_data_df_test = pd.read_pickle(f"data/models_pickles_new/ern_cov_fal_models_test.pkl")

crn_data_df_test = pd.read_pickle(f"data/models_pickles_new/crn_models_test.pkl")
crn_cov_fal2_data_df_test = pd.read_pickle(f"data/models_pickles_new/crn_cov_fal2_models_test.pkl")

display(crn_cov_fal2_data_df_test[crn_cov_fal2_data_df_test.isna().any(axis=1)])
crn_cov_fal2_data_df_test['e_LT_F2_C'] = crn_cov_fal2_data_df_test['e_LT_F2_C'].fillna(crn_cov_fal2_data_df_test['e_LT_F2_C'].mean())
display(crn_cov_fal2_data_df_test[crn_cov_fal2_data_df_test.isna().any(axis=1)])

,e_CRN,e_LT_F2_C,Age,Sex,Handness,RRQ,DASS-21 Stress,DASS-21 Dep,STAI-T,STAI-S Diff,...,T-CTR,OB_PERF,PS,AMB,PRED,STAND,IUS-P,IUS-I,SES,performance
98,0.000004,None,0.394473,0.0,1.0,4.17,3.0,2.14,2.65,-4.0,...,3.2,5.8,3.57,4.67,4.63,6.0,4.0,3.8,2.5,0.38665


,e_CRN,e_LT_F2_C,Age,Sex,Handness,RRQ,DASS-21 Stress,DASS-21 Dep,STAI-T,STAI-S Diff,...,T-CTR,OB_PERF,PS,AMB,PRED,STAND,IUS-P,IUS-I,SES,performance


In [10]:
test_datasets = [
    ern_data_df_test, 
    ern_cov_fal_data_df_test, 
    crn_data_df_test,
    crn_cov_fal2_data_df_test,
]

## Perform full bootstrap

In [11]:
def bootstrap_network(
    X, 
    model='adaptive_sklearn', 
    N=100, 
):
    bootstrapped_matrices = []
    n = len(X)
    for i in range(N):
        
        # Generate a bootstrap sample
        bootstrap_X = resample(X, n_samples=n, replace=True)
        print(f'{i} iteration')
        _, precision_matrix = estimate_graph(bootstrap_X)
        bootstrapped_matrices.append(precision_matrix)
            
    return bootstrapped_matrices

Perform bootstrapping

In [ ]:
logging.getLogger().setLevel(logging.INFO)

for index, dataset in enumerate(datasets):
    print(f'Estimating {index} dataset ######################')

    bootstraped_precision_matrices = bootstrap_network(
        X = dataset,
        N=1000,
    )

    with open(f'data/bootstrap_results/bootstrap_precision_matrices_{index}.pkl', 'wb') as f:
        pickle.dump(bootstraped_precision_matrices, f)

## Estimate the stability of network measures using bootstrapping

In [16]:
def calculate_nodes_predictability(X, G):
    explained_variance = dict()
        
    for node in G.nodes():
        y_ = X[[node]]

        neighbors = list(G.neighbors(node))

        X_ = X.loc[:, neighbors]

        lm = LinearRegression()
        lm.fit(X_, y_)

        score = lm.score(X_,y_)
        explained_variance[node] = score

    return explained_variance

In [17]:
def get_links(precision_matrix_df, threshold=0.02):
    precision_matrix_df = precision_matrix_df.where(np.triu(np.ones(precision_matrix_df.shape)).astype(bool))
    
    links = precision_matrix_df.stack().reset_index()
    links.columns = ['var1', 'var2','weight']
    links=links.loc[ (abs(links['weight']) > threshold) &  (links['var1'] != links['var2']) ]
        
    links = links.round(3)
    
    return links

In [18]:
def get_ranked_dict(dict_):
    items = [(key, value) for key, value in dict_.items()]
    sorted_items = sorted(items, key=lambda x: x[1], reverse=True)

    ranked_dict = {}
    rank = 1

    for key, value in sorted_items:
        ranked_dict[key] = rank
        rank += 1

    return ranked_dict

In [26]:
def bootstrap_network_measures(
    X, 
    model='adaptive_sklearn', 
    measures=None, 
    N=1000, 
    levels=np.arange(.95, 0.25, -0.05)
):
    results_df = pd.DataFrame()
    network_measures_baseline = []
    
    G_, _ = estimate_graph(X)
    
    for measure, measure_parameters in measures: 
        network_measure = measure(G = G_, **measure_parameters)
        ranked_network_measure = get_ranked_dict(network_measure)
        ranked_network_measure = {k: v for k, v in sorted(ranked_network_measure.items(), key=lambda item: item[0], reverse=True)}
        logging.info(ranked_network_measure)
        baseline_measure = list(ranked_network_measure.values())
        
        network_measures_baseline.append(baseline_measure)
        logging.info('Baseline measures appended')
    
    for level in levels:
        n = int(level * len(X))
        print(f'X len before samples: {n}')

        for i in range(N):
            # Generate a bootstrap sample with replacements
            bootstrap_X = resample(X, n_samples=n, replace=True)
            print(f'X len inside samples: {len(bootstrap_X)}')
            G_, _ = estimate_graph(bootstrap_X)
            
            for index, network_measure in enumerate(measures): 
                measure, measure_parameters = network_measure
                
                current_baseline = network_measures_baseline[index]
                
                try:
                    network_measure = measure(G = G_, **measure_parameters)
                    ranked_network_measure = get_ranked_dict(network_measure)
                    ranked_network_measure = {k: v for k, v in sorted(ranked_network_measure.items(), key=lambda item: item[0], reverse=True)}
                    logging.info(ranked_network_measure)
                    ranked_network_measure = list(ranked_network_measure.values())

                    try:
                        similarity_corr_coef, p_value = scipy.stats.pearsonr(current_baseline, ranked_network_measure)
                        print(f"Measure: {measure.__name__}  Level: {level}   sample: {i}   : similarity: {similarity_corr_coef}")
                        this_results = pd.DataFrame({
                            'measure': [measure.__name__],
                            'level': [level],
                            'similatity': [similarity_corr_coef],
                        })

                        results_df = pd.concat([results_df, this_results], ignore_index = True)
                    except:
                        logging.info('DIFFERENT LENGTHS OF BASELINE AND CURRENT')
                except:
                    logging.info('NETWORK MEASURE ERROR')
                    
            
    return results_df

Estimate the stability of network measures

In [27]:
logging.getLogger().setLevel(logging.INFO)

for index, (dataset_train, dataset_test) in enumerate(zip(datasets, test_datasets)):

    df_ranked = bootstrap_network_measures(
        X = dataset_train,
        measures = [
            (calculate_nodes_predictability, {'X': dataset_test}), 
            (nx.degree_centrality, {}),
            (nx.closeness_centrality, {}),
            (nx.current_flow_closeness_centrality, {'weight': 'weight'}),
            (nx.betweenness_centrality, {'weight': 'weight'}),
            (nx.current_flow_betweenness_centrality, {'weight': 'weight'}),
            (nx.load_centrality, {})
        ],
        N=2,
        levels=np.arange(.95, 0.25, -0.05)

    )
    # df_ranked.to_pickle(f'data/network_analysis/stability_estimates/network_measures_ranked_bootstrapped_{index}_pred_test.pkl')

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.4s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    2.0s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    5.3s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:   10.4s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:   13.1s finished
INFO:root:{'e_ERN': 32, 'WBSI': 19, 'WASH': 27, 'T-CTR': 16, 'STAND': 26, 'STAI-T': 1, 'STAI-S Diff': 31, 'SES': 3, 'RRQ': 11, 'REW': 17, 'PUN': 4, 'PS': 22, 'PRED': 21, 'OT': 10, 'ORD': 23, 'OB_PERF': 15, 'OBSESS': 13, 'NEU': 25, 'IUS-P': 12, 'IUS-I': 8, 'INDEC_F': 7, 'HOARD': 28, 'HARM': 30, 'G_SE': 24, 'DASS-21 Stress': 6, 'DASS-21 Dep': 5, 'CHECK': 29, 'BIS': 2, 'BAS_PRZY': 20, 'BAS_NAG': 18, 'BAS_D': 14, 'AMB': 9}
INFO:root:Baseline measures appended
INFO:root:{'e_ERN': 31, 'WBSI': 27, 'WASH': 13, 'T-CTR': 18, 'STAND': 26, 'STAI-T': 24, 'STAI-S Diff': 28, 'SES': 3, 'RRQ': 23, 'REW': 21

X len before samples: 247
X len inside samples: 247


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    1.0s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    5.4s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   12.5s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:   22.6s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:   28.8s finished
INFO:root:{'e_ERN': 31, 'WBSI': 19, 'WASH': 29, 'T-CTR': 20, 'STAND': 26, 'STAI-T': 1, 'STAI-S Diff': 28, 'SES': 3, 'RRQ': 13, 'REW': 18, 'PUN': 4, 'PS': 27, 'PRED': 23, 'OT': 7, 'ORD': 10, 'OB_PERF': 14, 'OBSESS': 11, 'NEU': 24, 'IUS-P': 17, 'IUS-I': 5, 'INDEC_F': 12, 'HOARD': 25, 'HARM': 30, 'G_SE': 22, 'DASS-21 Stress': 8, 'DASS-21 Dep': 6, 'CHECK': 32, 'BIS': 2, 'BAS_PRZY': 21, 'BAS_NAG': 15, 'BAS_D': 16, 'AMB': 9}
INFO:root:{'e_ERN': 3, 'WBSI': 22, 'WASH': 8, 'T-CTR': 23, 'STAND': 5, 'STAI-T': 31, 'STAI-S Diff': 2, 'SES': 29, 'RRQ': 18, 'REW': 16, 'PUN': 28, 'PS': 11, 'PRED': 12, 'OT':

Measure: calculate_nodes_predictability  Level: 0.95   sample: 0   : similarity: 0.9362170087976538
Measure: degree_centrality  Level: 0.95   sample: 0   : similarity: -0.06341642228739003
Measure: closeness_centrality  Level: 0.95   sample: 0   : similarity: 0.0835777126099707
Measure: current_flow_closeness_centrality  Level: 0.95   sample: 0   : similarity: 0.5626832844574781
Measure: betweenness_centrality  Level: 0.95   sample: 0   : similarity: 0.20197947214076245
Measure: current_flow_betweenness_centrality  Level: 0.95   sample: 0   : similarity: 0.47470674486803516
Measure: newman_betweenness_centrality  Level: 0.95   sample: 0   : similarity: 0.02639296187683285
X len inside samples: 247


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.5s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    2.5s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    5.8s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:   10.7s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:   13.6s finished
INFO:root:{'e_ERN': 32, 'WBSI': 14, 'WASH': 26, 'T-CTR': 16, 'STAND': 23, 'STAI-T': 1, 'STAI-S Diff': 31, 'SES': 4, 'RRQ': 13, 'REW': 18, 'PUN': 3, 'PS': 25, 'PRED': 21, 'OT': 8, 'ORD': 22, 'OB_PERF': 12, 'OBSESS': 11, 'NEU': 24, 'IUS-P': 19, 'IUS-I': 7, 'INDEC_F': 6, 'HOARD': 28, 'HARM': 29, 'G_SE': 27, 'DASS-21 Stress': 9, 'DASS-21 Dep': 5, 'CHECK': 30, 'BIS': 2, 'BAS_PRZY': 20, 'BAS_NAG': 15, 'BAS_D': 17, 'AMB': 10}
INFO:root:{'e_ERN': 32, 'WBSI': 13, 'WASH': 23, 'T-CTR': 3, 'STAND': 28, 'STAI-T': 26, 'STAI-S Diff': 12, 'SES': 21, 'RRQ': 25, 'REW': 27, 'PUN': 19, 'PS': 24, 'PRED': 7, 'OT

...

Measure: calculate_nodes_predictability  Level: 0.6499999999999997   sample: 0   : similarity: 0.9758179231863443
Measure: degree_centrality  Level: 0.6499999999999997   sample: 0   : similarity: 0.6266002844950215
Measure: closeness_centrality  Level: 0.6499999999999997   sample: 0   : similarity: 0.6228070175438598
Measure: current_flow_closeness_centrality  Level: 0.6499999999999997   sample: 0   : similarity: 0.7731152204836418
Measure: betweenness_centrality  Level: 0.6499999999999997   sample: 0   : similarity: 0.22783309625414896
Measure: current_flow_betweenness_centrality  Level: 0.6499999999999997   sample: 0   : similarity: 0.6877667140825036
Measure: newman_betweenness_centrality  Level: 0.6499999999999997   sample: 0   : similarity: 0.5403034613560931
X len inside samples: 168


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.8s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    3.4s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    7.8s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:   14.3s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:   18.1s finished
INFO:root:{'performance': 37, 'e_LT_F': 31, 'e_ERN': 29, 'WBSI': 16, 'WASH': 26, 'T-CTR': 18, 'Sex': 30, 'STAND': 24, 'STAI-T': 1, 'STAI-S Diff': 35, 'SES': 2, 'RRQ': 10, 'REW': 15, 'PUN': 3, 'PS': 23, 'PRED': 20, 'OT': 6, 'ORD': 21, 'OB_PERF': 14, 'OBSESS': 11, 'NEU': 25, 'IUS-P': 13, 'IUS-I': 9, 'INDEC_F': 12, 'Handness': 34, 'HOARD': 28, 'HARM': 32, 'G_SE': 27, 'DASS-21 Stress': 5, 'DASS-21 Dep': 8, 'CHECK': 33, 'BIS': 4, 'BAS_PRZY': 17, 'BAS_NAG': 22, 'BAS_D': 19, 'Age': 36, 'AMB': 7}
INFO:root:{'performance': 35, 'e_LT_F': 34, 'e_ERN': 13, 'WBSI': 24, 'WASH': 23, 'T-CTR': 18, 'Sex': 4,

Measure: calculate_nodes_predictability  Level: 0.6499999999999997   sample: 1   : similarity: 0.9770033191085824
Measure: degree_centrality  Level: 0.6499999999999997   sample: 1   : similarity: 0.4516358463726886
Measure: closeness_centrality  Level: 0.6499999999999997   sample: 1   : similarity: 0.4798482693219537
Measure: current_flow_closeness_centrality  Level: 0.6499999999999997   sample: 1   : similarity: 0.8539592223802752
Measure: betweenness_centrality  Level: 0.6499999999999997   sample: 1   : similarity: 0.027975343764817445
Measure: current_flow_betweenness_centrality  Level: 0.6499999999999997   sample: 1   : similarity: 0.7965860597439548
Measure: newman_betweenness_centrality  Level: 0.6499999999999997   sample: 1   : similarity: 0.44073020388809864
X len before samples: 155
X len inside samples: 155


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.7s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    3.7s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:    8.5s
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:   15.7s


KeyboardInterrupt: 